# RocketPy Simulation
This notebook was generated using Rocket-Serializer, a RocketPy tool to convert simulation files to RocketPy simulations
The notebook was generated using the following parameters file: `examples/NDRT--Rocket--2020/parameters.json`


In [ ]:
%pip install rocketpy<=2.0

In [ ]:
from rocketpy import (
    Environment,
    SolidMotor,
    Rocket,
    Flight,
    TrapezoidalFins,
    RailButtons,
    NoseCone,
    Tail,
)
import datetime

## Environment


In [ ]:
env = Environment()
env.set_location(latitude=28.61, longitude=-80.6)
env.set_elevation(0.0)

Optionally, you can set the date and atmospheric model


In [ ]:
tomorrow = datetime.date.today() + datetime.timedelta(days=1)
env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))
env.set_atmospheric_model(type="Forecast", file="GFS")

In [ ]:
env.all_info()

## Motor
Currently, only Solid Motors are supported by Rocket-Serializer
If you want to use a Liquid or Hybrid motor, please use rocketpy directly.


In [ ]:
motor = SolidMotor(
    thrust_source="thrust_source.csv",
    dry_mass=0,
    center_of_dry_mass_position=0,
    dry_inertia=[0, 0, 0],
    grains_center_of_mass_position=0,
    grain_number=1,
    grain_density=1202.7971321820446,
    grain_outer_radius=0.0375,
    grain_initial_inner_radius=0.01875,
    grain_initial_height=0.621,
    grain_separation=0,
    nozzle_radius=0.028124999999999997,
    nozzle_position=-0.3105,
    throat_radius=0.01875,
    reshape_thrust_curve=False,  # Not implemented in Rocket-Serializer
    interpolation_method="linear",
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

In [ ]:
motor.all_info()

## Rocket
Currently, only single stage rockets are supported by Rocket-Serializer
We will start by defining the aerodynamic surfaces, and then build the rocket.


### Nosecones


In [ ]:
nosecone = NoseCone(
    length=0.6095999999999999,
    kind="ogive",
    base_radius=0.1016,
    rocket_radius=0.1016,
    name="0.6095999999999999",
)

### Fins
As rocketpy allows for multiple fins sets, we will create a dictionary with all the fins sets and then add them to the rocket


In [ ]:
trapezoidal_fins = {}

In [ ]:
trapezoidal_fins[0] = TrapezoidalFins(
    n=4,
    root_chord=0.15239999999999995,
    tip_chord=0.07619999999999999,
    span=0.1651,
    cant_angle=0.0,
    sweep_length=0.038099999999999995,
    sweep_angle=None,
    rocket_radius=0.1016,
    name="Trapezoidal fin set",
)

### Transitions (Tails)
As rocketpy allows for multiple tails, we will create a dictionary with all the tails and then add them to the rocket


In [ ]:
tails = {}

In [ ]:
tails[0] = Tail(
    top_radius=0.1016,
    bottom_radius=0.0776224,
    length=0.127,
    rocket_radius=0.1016,
    name="Transition",
)

In [ ]:
rocket = Rocket(
    radius=0.1016,
    mass=20.735,
    inertia=[0.15089, 0.15089, 18.411],
    power_off_drag="drag_curve.csv",
    power_on_drag="drag_curve.csv",
    center_of_mass_without_motor=1.7041,
    coordinate_system_orientation="nose_to_tail",
)

### Adding surfaces to the rocket
Now that we have all the surfaces, we can add them to the rocket


In [ ]:
rocket.add_surfaces(
    surfaces=[nosecone, trapezoidal_fins[0], tails[0]],
    positions=[0.0, 3.3528000000000002, 1.1938],
)

In [ ]:
rocket.add_motor(motor, position=3.079819999999998)

In [ ]:
### Rocket Info
rocket.all_info()

## Flight
We will now create the flight simulation. Let's go!


In [ ]:
flight = Flight(
    rocket=rocket,
    environment=env,
    rail_length=2.7432000000000003,
    inclination=85.0,
    heading=90.0,
    terminate_on_apogee=False,
    max_time=600,
)

In [ ]:
flight.all_info()